## How does this script work?

#### Part 1:
- scrape all the available items from strategics lululemon website
- make an excel sheet for people to fill out containing drop downs of all the items 

#### Part 2:
- gather all the orders filled out by the team. Group by item name, colour, size and quantity

#### Part 3:
- attach a SKU to P2
- use output from part 2 to insert items into cart


In [8]:
# !pip install lxml
# !pip install BeautifulSoup4
# !pip install tqdm
# !pip install google-sheets-to-csv

Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\program files (x86)\python37-32\python.exe -m pip install --upgrade pip' command.


In [3]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import requests
from lxml import html
from bs4 import BeautifulSoup 

import time
from datetime import datetime
from time import sleep, localtime, strftime

import json 
import numpy as np
import pandas as pd 
from tqdm import tqdm

import os
import os.path
import re

from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--enable-javascript")
# chrome_options.add_argument("--headless")
# driver = webdriver.Chrome(options=chrome_options)

In [27]:
# initiate website webdriver and navigate to portal
driver = webdriver.Chrome('./chromedriver.exe', options = chrome_options)
driver.set_window_size(1920, 960)

In [96]:
# set user credentials
def sign_in():
    
    user_ = username
    pass_ = password

    driver.get('https://strategicsales.lululemon.com/ca')

    # login with credentials
    username = driver.find_element_by_name('ShopLoginForm_Login')
    username.send_keys(user_)

    password = driver.find_element_by_name('ShopLoginForm_Password')
    time.sleep(2)
    password.send_keys(pass_)

    sign_in = driver.find_element_by_name('login')
    sign_in.click()

    wait = WebDriverWait(driver, 10)
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'caret')))

    # select an address
    addressdropdown_xpath= '//*[@id="shipto-form"]/div[1]/div/div/button/span[2]/span'
    address_xpath='//*[@id="shipto-form"]/div[1]/div/div/div/ul/li[4]/a/span[2]'
    address_continue_xpath = '//*[@id="shipto-form"]/div[2]/button'

    sleep(2)
    driver.find_element_by_xpath(addressdropdown_xpath).click()
    sleep(1)
    driver.find_element_by_xpath(address_xpath).click()
    sleep(1)
    driver.find_element_by_xpath(address_continue_xpath).click()

In [97]:
sign_in()

## 1. Get all items currently in stock

In [3]:
def get_item_name_price(pagination_xpath:str, items=['Womens', 'Mens']):
    
    gender_names_prices = []
    
    for gender in items:

        if gender == 'Womens':

            womens_xpath = '//*[@id="globalnav"]/div/div/div[1]/ul/li[2]/a[1]'
            driver.find_element_by_xpath(womens_xpath).click()
            wait.until(EC.presence_of_element_located((By.ID, 'product-search-result')))
            print('Found element...')

        elif gender == 'Mens':

            mens_xpath = '//*[@id="globalnav"]/div/div/div[1]/ul/li[3]/a[1]'
            driver.find_element_by_xpath(mens_xpath).click()
            wait.until(EC.presence_of_element_located((By.ID, 'product-search-result')))
            print('Found element...')
        
        print(f'Retrieving data for {gender} items.')
        
        sleep(2)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source)

        sleep(2)
        max_page = sorted(list(set(soup.find(class_='pagination-site-list').get_text(strip = True)\
        .replace('to page', '').replace('of', '').replace('«', '').replace('»', ''))))[-1]

        print(f'Max pages for {gender} items is {int(max_page)+1}.')

        for i in tqdm(range(1, int(max_page) + 2)):
            
            print('Now on page: ', i)

            wait.until(EC.presence_of_element_located((By.ID, 'product-search-result')))
            # wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'view-all')))
            page_source = driver.page_source
            soup = BeautifulSoup(page_source)
            

            for name, price in list(zip(soup.find_all(class_='product-title'), 
                                              soup.find_all(class_='wholesale-price'))):

                gender_names_prices.append(((gender),
                                            name.get_text(strip = True), 
                                            price.get_text(strip = True).replace('Wholesale Price:$', ''),
                                            name['href']))


            if i == int(max_page)+1:
                break
            else:
                retry_attempts = 0
                while retry_attempts < 5:
                    try:
                        wait.until(EC.presence_of_element_located((By.ID, 'product-search-result')))
                        # wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'view-all')))
                        driver.find_element_by_xpath(pagination_xpath).click()
                        sleep(2)
                        break
                    except:
                        print('Could not find next page link. Retrying...')
                        retry_attempts +=1
                        print(retry_attempts)

            # print('Moving to next page...')
        
    print('All item data retrieved.')
    
    item_df = pd.DataFrame(data = gender_names_prices, columns = ['Gender', 'Item_name', 'Price', 'link'])
    item_df_dd = item_df.drop_duplicates().reset_index(drop=True)
    
    from time import localtime
    localtime = (strftime("%Y-%m-%d %H:%M:%S", localtime()))  
    localtime_trf = localtime.replace('-', '_').replace(':', '_')
    
    item_df_dd.to_csv(f'Current_selection_luluelemon_items_{items[0]}_{localtime_trf}.csv')
    
    # driver.quit()
    
    return item_df_dd

In [15]:
%%time
womens_items = get_item_name_price('//*[@id="categoryPaginationOnly"]/div/div/div[2]/ul/li[12]/a', items=['Womens'])
mens_items = get_item_name_price('//*[@id="categoryPagination"]/div/div[1]/div[2]/ul/li[9]/a', items=['Mens'])

Found element...
Retrieving data for Womens items.
Max pages for Womens items is 10.


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Now on page:  1


 10%|████████▎                                                                          | 1/10 [00:02<00:21,  2.36s/it]

Now on page:  2


...

Now on page:  8


 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [00:18<00:04,  2.37s/it]

Now on page:  9


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:21<00:02,  2.38s/it]


Now on page:  10
All item data retrieved.
Wall time: 28 s


## 2. Get SKUs per item colour and size combination

In [19]:
mens_items = pd.read_csv('Current_selection_luluelemon_items_Mens_2022_08_01 00_26_36.csv').drop('Unnamed: 0', axis=1)
womens_items = pd.read_csv('Current_selection_luluelemon_items_Womens_2022_08_01 00_21_23.csv').drop('Unnamed: 0', axis=1)

In [342]:
def get_inventoryinfo(df, gender:str):
   
    df_main = pd.DataFrame(columns = ['Item', 'Type', 'Colour', 'Size', 'Price', 'InventoryAvailability', 'SKU'])
    
    gender_name_prices = df.values.tolist()
    counter = 0
    
    try:
        for name in tqdm(gender_name_prices):

            try:
                print(f'Entering page for {name[1]}, item num: {counter}')
            #         wait.until(EC.presence_of_element_located((By.ID, 'product-search-result')))
            #         wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'view-all')))
                time.sleep(2)
                driver.get(name[3])
                print('Finding element on page...')
                wait.until(EC.presence_of_element_located((By.ID, 'addgroupproductbutton')))
                print('Element found...')
                print('Extracting HTML...')
                item_page_source = driver.page_source
                item_soup = BeautifulSoup(item_page_source)

                # get a list of SKUs
                SKU_list = []
                for i in item_soup.find_all('input'):

                    # print(i)
                    if i['name']=='SKU':
                        SKU_list.append(i['value'])

                # SKU_list
                table = item_soup.find_all('table')
                df = pd.read_html(str(table))[0]
                df.columns = df.columns.droplevel()
                df = df.iloc[:-1, 1:-1]

                col_list = df.columns.tolist()
                col_list[0] = 'Colour'
                col_list[1] = 'Availability'
                
                # used to retain order of cols
                if 'S' in col_list or 'M' in col_list or 'L' in col_list:
                    col_list = [f'{y}' + x if x != 'Colour' and x != 'Availability' else x 
                                for x, y in list(zip(col_list, range(len(col_list))))]
                    
                if '10' in col_list or '12' in col_list or '14' in col_list:
                    col_list = [f'{y}' + x if x != 'Colour' and x != 'Availability' and y <= 9 
                                else '9' + x if x != 'Colour' and x != 'Availability' else x   
                                for x, y in list(zip(col_list, range(len(col_list))))]
                    
                col_list = ['Inventory' + x if x != 'Colour' else x for x in col_list]
                df.columns = col_list

                df1 = []
                for i in df.Colour.values:

                    df1.append(pd.wide_to_long(df[df['Colour']==i], 
                                stubnames = ['Availability', 'Inventory'], 
                                i = 'Colour', j = 'Size', suffix=r'\w+'))

                df=pd.concat(df1)

                df['Inventory'] = df['Inventory'].fillna(0.0).replace('Available', 0.0)
                df['InventoryAvailability'] = df['Inventory'].apply(lambda x: ['Not Available' if x == 0.0 else 'Available'][0])

                # add item name as column
                df = df.reset_index()
                df['Type'] = name[0]
                df['Price'] = name[2]
                df['Item'] = name[1]
                cols = list(df.columns)
                cols = [cols[-1]] + cols[:-1]
                df = df[cols]

                # filter out items not Availabe
                df = df[df['InventoryAvailability'] != 'Not Available']

                # Add SKU per item

                print(f'Adding SKUs for {name[1]}')
                df['SKU'] = SKU_list[:df.shape[0]]
                if len([x[0] for x in [re.findall(r"([1-9]|[0-1][0-2])[A-Z]{1}", x) for x in col_list] if x != []]) != 0:
                    # df['Size'] = df['Size'].apply(lambda x: [x[1:] if len(re.findall(r"([1-9]|[0-1][0-2])[A-Z]{1}", x)[0])<= 1 else x[2:]][0])
                    df['Size'] = df['Size'].apply(lambda x: x[1:])
                if len([x[0] for x in [re.findall(r"\d{2}", x) for x in col_list] if x != []]) != 0:
                    # df['Size'] = df['Size'].apply(lambda x: [x[1:] if len(re.findall(r"\d{2}", x)[0])<=1 else x[2:]][0])
                    if gender == 'mens':
                        df['Size'] = df['Size'].apply(lambda x: x[1:] if '9' in x else x)
                    else:
                        df['Size'] = df['Size'].apply(lambda x: x[1:])
                        
                df_main = pd.concat([df, df_main])
                

                print(f'Successfully added info for {name[1]}')
                counter +=1

        #         driver.back()
        #         wait.until(EC.presence_of_element_located((By.ID, 'product-search-result')))
        #         print('Found element...moving to next item')
            except Exception as e:
                print(e)
                pass
    
    except Exception as e:
        print(e)
        pass

    from time import localtime
    localtime = (strftime("%Y-%m-%d %H:%M:%S", localtime()))  
    localtime_trf = localtime.replace('-', '_').replace(':', '_')

    # df_main = df_main.drop('Availability', axis=1)
    df_main.to_csv(f'inventoryinfo_{gender}.csv')

    return df_main

In [332]:
# df_main = pd.DataFrame(columns = ['Item', 'Type', 'Colour', 'Size', 'Price', 'InventoryAvailability', 'SKU'])

# gender_name_prices = mens_items[0:1].values.tolist()
# counter = 0
# gender = 'Mens'

# for name in tqdm(gender_name_prices):

#     print(f'Entering page for {name[1]}, item num: {counter}')
# #         wait.until(EC.presence_of_element_located((By.ID, 'product-search-result')))
# #         wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'view-all')))
#     time.sleep(2)
#     driver.get(name[3])
#     print('Finding element on page...')
#     wait.until(EC.presence_of_element_located((By.ID, 'addgroupproductbutton')))
#     print('Element found...')
#     print('Extracting HTML...')
#     item_page_source = driver.page_source
#     item_soup = BeautifulSoup(item_page_source)

#     # get a list of SKUs
#     SKU_list = []
#     for i in item_soup.find_all('input'):

#         # print(i)
#         if i['name']=='SKU':
#             SKU_list.append(i['value'])
            
#     # SKU_list
#     table = item_soup.find_all('table')
#     df = pd.read_html(str(table))[0]
#     df.columns = df.columns.droplevel()
#     df = df.iloc[:-1, 1:-1]
    
#     col_list = df.columns.tolist()
#     col_list[0] = 'Colour'
#     col_list[1] = 'Availability'
# #     if 'S' in col_list or 'M' in col_list or 'L' in col_list:
# #         col_list = [f'{y}' + x if x != 'Colour' and x != 'Availability' else x for x, y in list(zip(col_list, range(len(col_list))))]
        
# #     if '10' in col_list or '12' in col_list or '14' in col_list:
# #         col_list = [f'{y}' + x if x != 'Colour' and x != 'Availability' else x for x, y in list(zip(col_list, range(len(col_list))))]

#     if 'S' in col_list or 'M' in col_list or 'L' in col_list:
#         col_list = [f'{y}' + x if x != 'Colour' and x != 'Availability' else x 
#                     for x, y in list(zip(col_list, range(len(col_list))))]

#     if '10' in col_list or '12' in col_list or '14' in col_list or '30' in col_list or :
#         col_list = [f'{y}' + x if x != 'Colour' and x != 'Availability' and y <= 9 
#                     else '9' + x if x != 'Colour' and x != 'Availability' else x   
#                     for x, y in list(zip(col_list, range(len(col_list))))]

#     col_list = ['Inventory' + x if x != 'Colour' else x for x in col_list]
#     df.columns = col_list

#     df1 = []
#     for i in df.Colour.values:

#         df1.append(pd.wide_to_long(df[df['Colour']==i], 
#                     stubnames = ['Availability', 'Inventory'], 
#                     i = 'Colour', j = 'Size', suffix=r'\w+'))

#     df=pd.concat(df1)

#     df['Inventory'] = df['Inventory'].fillna(0.0).replace('Available', 0.0)
#     df['InventoryAvailability'] = df['Inventory'].apply(lambda x: ['Not Available' if x == 0.0 else 'Available'][0])

#     # add item name as column
#     df = df.reset_index()
#     df['Type'] = name[0]
#     df['Price'] = name[2]
#     df['Item'] = name[1]
#     cols = list(df.columns)
#     cols = [cols[-1]] + cols[:-1]
#     df = df[cols]

#     # filter out items not Availabe
#     df = df[df['InventoryAvailability'] != 'Not Available']

#     # Add SKU per item

# print(f'Adding SKUs for {name[1]}')
# df['SKU'] = SKU_list[:df.shape[0]]
# df_main = pd.concat([df, df_main])
# if 'Inventory2S' in col_list or 'Inventory3M' in col_list or 'Inventory4L' in col_list:
#     df_main['Size'] = df_main['Size'].apply(lambda x: x[1:])
# if 'Inventory22' in col_list or 'Inventory45' in col_list:
#     df_main['Size'] = df_main['Size'].apply(lambda x: x[1:])

# print(f'Successfully added info for {name[1]}')

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Entering page for ABC Pant Classic-Fit *Warpstreme 32", item num: 0
Finding element on page...
Element found...
Extracting HTML...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.69s/it]


In [338]:
%%time
mens_inventoryinfo = get_inventoryinfo(mens_items, gender='mens')
womens_inventoryinfo = get_inventoryinfo(womens_items, gender='womens')
mens_inventoryinfo

  0%|                                                                                           | 0/94 [00:00<?, ?it/s]

Entering page for ABC Pant Classic-Fit *Warpstreme 32", item num: 0
Finding element on page...
Element found...
Extracting HTML...


  1%|▉                                                                                  | 1/94 [00:03<05:36,  3.62s/it]

Adding SKUs for ABC Pant Classic-Fit *Warpstreme 32"
Successfully added info for ABC Pant Classic-Fit *Warpstreme 32"
Entering page for ABC Pant Classic-Fit *Warpstreme 34", item num: 1
Finding element on page...
Element found...
Extracting HTML...


  2%|█▊                                                                                 | 2/94 [00:07<06:11,  4.03s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████████▏| 93/94 [05:02<00:03,  3.02s/it]

Element found...
Extracting HTML...
Adding SKUs for Vented Tennis Short
Successfully added info for Vented Tennis Short
Entering page for Vented Tennis Short Sleeve, item num: 93
Finding element on page...


100%|██████████████████████████████████████████████████████████████████████████████████| 94/94 [05:05<00:00,  3.25s/it]

Element found...
Extracting HTML...
Adding SKUs for Vented Tennis Short Sleeve
Successfully added info for Vented Tennis Short Sleeve
Wall time: 5min 5s


,Item,Colour,Size,Availability,Inventory,InventoryAvailability,Type,Price,SKU,InventoryONE SIZE
0,Vented Tennis Short Sleeve,White,S,NaN,10.0,Available,Mens,49.0,142325963,NaN
1,Vented Tennis Short Sleeve,White,M,NaN,43.0,Available,Mens,49.0,142325958,NaN
2,Vented Tennis Short Sleeve,White,L,NaN,36.0,Available,Mens,49.0,142325951,NaN
3,Vented Tennis Short Sleeve,White,XL,NaN,18.0,Available,Mens,49.0,142325970,NaN
4,Vented Tennis Short Sleeve,White,XXL,NaN,4.0,Available,Mens,49.0,142325983,NaN
...,...,...,...,...,...,...,...,...,...,...
54,"ABC Pant Classic-Fit *Warpstreme 32""",Silver Drop,48,NaN,10.0,Available,Mens,69.0,136302986,NaN
65,"ABC Pant Classic-Fit *Warpstreme 32""",True Navy,42,NaN,7.0,Available,Mens,69.0,136500065,NaN
66,"ABC Pant Classic-Fit *Warpstreme 32""",True Navy,44,NaN,8.0,Available,Mens,69.0,136500066,NaN
67,"ABC Pant Classic-Fit *Warpstreme 32""",True Navy,46,NaN,4.0,Available,Mens,69.0,136500067,NaN


In [346]:
womens_inventoryinfo.to_csv('inventoryinfo_womens.csv')

## 3. Get items ordered from google sheet

In [23]:
# navigate to google sheet url; download Orders Sheet
sheet_id = '15ZL3WeDxM4a-WCwhcJbGCYehppWiLLUF8VIgOIjTdt4'
sheet_name = 'Orders Sheet'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [28]:
driver.get(url)

In [286]:
path = 'C:/Users/ernes/Downloads/data.csv'
test = pd.read_csv(path)

cols = ['Item Name (Please Put Down Name As Seen On Website)', 
        'Colour', 
        'Size', 
        'Alt Colour (If Applicable)', 
        'Quantity',
        'Cost (As Listed)']

df_order = test[cols]

In [287]:
def clean(df):
    
    df = df.apply(lambda x: x.astype(str).str.upper())
    df = df.replace('NAN', '')
    
    try:
        df['Cost (As Listed)'] = df['Cost (As Listed)'].apply(lambda x: x.replace('$', ''))
    except:
        df['Price'] = df['Price'].apply(lambda x: x.replace('$', ''))
    
    return df

df_order = clean(df_order)
df_order.columns = ['Item', 'Colour', 'Size', 'AltColour', 'Quantity', 'Price']
df_order.head()

,Item,Colour,Size,AltColour,Quantity,Price
0,"SURGE JOGGERS 29""",BLACK,M,,1,69.00
1,"ABC PANT CLASSIC FIT *WARPSTREME 32""",BLACK,33,,2,69.00
2,CITY SWEAT FULL ZIP HOODIE,BLACK,M,NAVY,2,64.00
3,"ABC JOGGER *WARPSTREME 30"" *",BLACK,M,,2,69.00
4,"THE SHORT 9"" LINERLESS",BLACK,M,,1,34.00


In [288]:
mens = pd.read_csv('inventoryinfo_mens.csv')
mens = clean(mens)
mens = mens[['Item', 'Colour', 'Size', 'SKU', 'Price', 'Inventory']]

womens = pd.read_csv('inventoryinfo_womens.csv')
womens = clean(womens)
womens = womens[['Item', 'Colour', 'Size', 'SKU', 'Price', 'Inventory']]

all_items = pd.concat([mens, womens])
all_items.head()

,Item,Colour,Size,SKU,Price,Inventory
0,VENTED TENNIS SHORT SLEEVE,WHITE,S,142325963,49.0,10.0
1,VENTED TENNIS SHORT SLEEVE,WHITE,M,142325958,49.0,43.0
2,VENTED TENNIS SHORT SLEEVE,WHITE,L,142325951,49.0,36.0
3,VENTED TENNIS SHORT SLEEVE,WHITE,XL,142325970,49.0,18.0
4,VENTED TENNIS SHORT SLEEVE,WHITE,XXL,142325983,49.0,4.0


In [289]:
all_items.columns

Index(['Item', 'Colour', 'Size', 'SKU', 'Price', 'Inventory'], dtype='object')

In [290]:
#  def clean_colour(colour):
 
def clean_size(size):
    
    if size.startswith('M'):
        return 'M'
    elif size.startswith('S'):
        return 'S'
    elif size.startswith('L'):
        return 'L'
    else:
        return size
    

In [291]:
# Apply preprocessing steps to size and groupby sum quantity
df_order['Size'].apply(lambda x: clean_size(x))
df_order = df_order.groupby(['Item', 'Colour', 'Size']).sum()
df_order = df_order.reset_index()
df_order.columns = ['Item', 'Colour', 'Size', 'AltColour','Quantity', 'Price']
df_order.head()

,Item,Colour,Size,AltColour,Quantity,Price
0,"ABC JOGGER *WARPSTREME 30"" *",BLACK,M,,2,69.00
1,"ABC PANT CLASSIC FIT *WARPSTREME 32""",BLACK,33,,2,69.00
2,CITY SWEAT FULL ZIP HOODIE,BLACK,M,NAVY,2,64.00
3,"LICENSE TO TRAIN SHORT 7"" LINERLESS",BLACK,M,,1,44.00
4,"SURGE JOGGERS 29""",BLACK,M,,1,69.00


## Challenges
1. Getting the exact match is difficult. We need another method to get the closest match if no exact match exists

In [292]:
import difflib
import numpy as np
from difflib import SequenceMatcher

def match_sku(df, df_sku):

    allitems = df_sku.Item.unique().tolist()
    
    item_list = []
    similarity_list = []
    colour_list = []
    sku = []
    list_price = []
    ordered_price = []
    notes = []
    quantity_avail = []

    for item, colour, size, altcolour, price in list(zip(df.Item.tolist(), 
                                       df.Colour.tolist(), 
                                       df.Size.tolist(),
                                       df.AltColour.tolist(),
                                       df.Price.tolist()         
                                      ) 
                                  ):

        match = difflib.get_close_matches(item, allitems, n=1)[0]
        
        # insert similarity matcher
        similarity = round(SequenceMatcher(None, item, match).ratio()*100, 2) 
        
        # print(item, match)
        
        try:
            
            sku.append(df_sku[(df_sku['Item']==match) & 
                            (df_sku['Colour']==colour) &
                            (df_sku['Size']==size)]['SKU'].values.tolist()[0]
                      )
            
            ordered_price.append(df_sku[(df_sku['Item']==match) & 
                            (df_sku['Colour']==colour) &
                            (df_sku['Size']==size)]['Price'].values.tolist()[0]
                        )

            quantity_avail.append(df_sku[(df_sku['Item']==match) & 
                            (df_sku['Colour']==colour) &
                            (df_sku['Size']==size)]['Inventory'].values.tolist()[0]
                        )
            
            colour_list.append(colour)
            notes.append('')
            list_price.append(price)
            item_list.append(match)
            similarity_list.append(similarity)
            
        except:
            
            try:
                if altcolour != '':
                    
                    sku.append(df_sku[(df_sku['Item']==match) & 
                                    (df_sku['Colour']==altcolour) &
                                    (df_sku['Size']==size)]['SKU'].values.tolist()[0]
                              )

                    ordered_price.append(df_sku[(df_sku['Item']==match) & 
                                    (df_sku['Colour']==altcolour) &
                                    (df_sku['Size']==size)]['Price'].values.tolist()[0]
                                ) 
    
                    quantity_avail.append(df_sku[(df_sku['Item']==match) & 
                                    (df_sku['Colour']==altcolour) &
                                    (df_sku['Size']==size)]['Inventory'].values.tolist()[0]
                                )
                    
                    colour_list.append(altcolour)
                    notes.append(f'{altcolour} used')
                    list_price.append(price)
                    item_list.append(match)
                    similarity_list.append(similarity)
                    
                else:
                    pass
                
            except:
                sku.append(np.nan)
                ordered_price.append(np.nan)
                colour_list.append(np.nan)
                notes.append(np.nan)
                list_price.append(np.nan)
                quantity_avail.append(np.nan)
                item_list.append(match)
                similarity_list.append(similarity)
    
    df['Matched_Item'] = item_list
    df['Matched_Similarity'] = similarity_list
    df['SKU'] = sku
    df['Listed_Price'] = list_price
    df['Ordered_Price'] = ordered_price
    df['ColourOrdered'] = colour_list
    df['Notes'] = notes
    df['Inventory'] = quantity_avail
    
    
    # calc total of order
    quant = df.Quantity.tolist()
    list_pric = df.Listed_Price.replace(np.nan, 0).tolist()
    ord_pric = df.Ordered_Price.replace(np.nan, 0).tolist()
    
    df['Row_Total_Listed'] = [float(a)*float(b) for a,b in zip(quant,list_pric)]
    df['Row_Total_Ordered'] = [float(a)*float(b) for a,b in zip(quant,ord_pric)]
    
    print('(Listed) Total Order Pre-Tax is: ', df['Row_Total_Listed'].sum())
    print('(Ordered) Total Order Pre-Tax is: ', df['Row_Total_Ordered'].sum())
    
    # calc if there's enough inventory for given quantity
    df['Enough_Quantity'] = ['Yes' if float(x) <= float(y) else 'No' for x, y in zip(quant,quantity_avail)]
    
    
    return df[['Item', 'Matched_Item', 
               'Matched_Similarity', 'SKU',
               'Colour', 'AltColour', 'ColourOrdered', 'Notes',
               'Size',
               'Quantity', 'Inventory',
               'Price', 'Listed_Price', 'Ordered_Price',
               'Enough_Quantity',
               'Row_Total_Listed', 'Row_Total_Ordered']]

In [293]:
df_order = match_sku(df_order, all_items)

(Listed) Total Order Pre-Tax is:  423.0
(Ordered) Total Order Pre-Tax is:  418.0


In [294]:
df_order

,Item,Matched_Item,Matched_Similarity,SKU,Colour,AltColour,ColourOrdered,Notes,Size,Quantity,Inventory,Price,Listed_Price,Ordered_Price,Enough_Quantity,Row_Total_Listed,Row_Total_Ordered
0,"ABC JOGGER *WARPSTREME 30"" *","ABC JOGGER *WARPSTREME 30""",96.30,114850041,BLACK,,BLACK,,M,2,125.0,69.00,69.00,69.0,Yes,138.0,138.0
1,"ABC PANT CLASSIC FIT *WARPSTREME 32""","ABC PANT CLASSIC-FIT *WARPSTREME 32""",97.22,136300434,BLACK,,BLACK,,33,2,11.0,69.00,69.00,69.0,Yes,138.0,138.0
2,CITY SWEAT FULL ZIP HOODIE,CITY SWEAT FULL ZIP HOODIE (S-5XL),86.67,NaN,BLACK,NAVY,NaN,NaN,M,2,NaN,64.00,NaN,NaN,No,0.0,0.0
3,"LICENSE TO TRAIN SHORT 7"" LINERLESS","LICENSE TO TRAIN SHORT 7"" *LINERLESS",98.59,122925085,BLACK,,BLACK,,M,1,106.0,44.00,44.00,44.0,Yes,44.0,44.0
4,"SURGE JOGGERS 29""","SURGE JOGGER 29""",96.97,122925054,BLACK,,BLACK,,M,1,126.0,69.00,69.00,64.0,Yes,69.0,64.0
5,"THE SHORT 9"" LINERLESS","T.H.E. SHORT 9"" *LINERLESS",91.67,3810740,BLACK,,BLACK,,M,1,30.0,34.00,34.00,34.0,Yes,34.0,34.0


## 3. Save SKUs and Quantity

In [165]:
df_order[['SKU', 'Quantity']][~df_order['SKU'].isnull()].to_csv('test.csv', index=False)

In [295]:
df_order[['SKU', 'Quantity']][~df_order['SKU'].isnull()]

,SKU,Quantity
0,114850041,2
1,136300434,2
3,122925085,1
4,122925054,1
5,3810740,1


## 4. Add to Cart via CSV Upload (Manual)
- lululemon has an option where if you have the SKU and quantity, you can upload that csv file directly to your cart